In [21]:
from utils import read_images
import pandas as pd
import numpy as np
import tifffile
import pyclesperanto_prototype as cle  # version 0.24.1
import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.6
from skimage import measure
from skimage.color import rgb2gray
from apoc import ObjectSegmenter, ObjectClassifier
from scipy import ndimage as ndi
import napari

In [8]:
# Define the directory containing your files
directory_path ="../data/test_data"

# The following function will read all the images contained within the directory_path above
# and store them grouped by well_id.
images_per_well = read_images(directory_path)

In [ ]:
# This is just a check, it prints the grouped z-stacks
for well_id, files in images_per_well.items():
    print(f'Well {well_id}:')
    for file_path in files:
        print(file_path)

In [12]:
imgs_to_process = images_per_well["A01"]

all_in_focus_labels = []  # Store in-focus labels for merging

for input_img in imgs_to_process:

    # Load one RGB image and transform it into grayscale (if needed) for APOC
    rgb_img = tifffile.imread(input_img, is_ome=False)
    if len(rgb_img.shape) < 3:
        img = rgb_img
    elif rgb_img.shape[2] == 3:
        img = rgb2gray(rgb_img)
    else:
        print(
            "Modify the loader to accommodate different file formats",
            rgb_img.shape,
        )

    # Apply object segmenter from APOC
    try:
        segmenter = ObjectSegmenter(opencl_filename="../ObjectSegmenter.cl")
        result = segmenter.predict(image=img)
    except IndexError:
        segmenter = ObjectSegmenter(
            opencl_filename="../pretrained_APOC/ObjectSegmenter.cl"
        )
        result = segmenter.predict(image=img)

    # Closing some holes in the organoid labels
    closed_labels = cle.closing_labels(result, None, radius=4.0)

    # Exclude small labels, cutout in pixel area seems to be below 1000px
    exclude_small = cle.exclude_small_labels(closed_labels, None, 1000.0)
    exclude_small = np.array(
        exclude_small, dtype=np.int32
    )  # Change dtype of closed labels to feed array into nsbatwm.split

    # Splitting organoids into a binary mask
    split_organoids = nsbatwm.split_touching_objects(exclude_small, sigma=10.0)

    # Connected component (cc) labeling
    cc_split_organoids = nsbatwm.connected_component_labeling(
        split_organoids, False
    )

    # Apply object classifier from APOC
    try:
        classifier = ObjectClassifier(opencl_filename="../ObjectClassifier.cl")
        result = classifier.predict(labels=cc_split_organoids, image=img)
    except AttributeError:
        classifier = ObjectClassifier(
            opencl_filename="../pretrained_APOC/ObjectClassifier.cl"
        )
        result = classifier.predict(labels=cc_split_organoids, image=img)

    # Convert the resulting .cle image into a np.array to count objects within each class
    image_array = np.array(result, dtype=np.int8)

    # Create masks for each class
    background_mask = image_array == 0
    out_of_focus_mask = image_array == 1
    in_focus_mask = image_array == 2

    # Label connected components in each mask
    background_labels = measure.label(background_mask, connectivity=2)
    out_of_focus_labels = measure.label(out_of_focus_mask, connectivity=2)
    in_focus_labels = measure.label(in_focus_mask, connectivity=2)
    
    # Store in-focus labels for merging
    all_in_focus_labels.append(in_focus_labels)



In [18]:
all_in_focus_labels[0].dtype

dtype('int32')

In [29]:
viewer = napari.Viewer(ndisplay=2)
for in_focus_label in all_in_focus_labels:
    viewer.add_labels(in_focus_label, name="In-focus organoids")

In [31]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(all_in_focus_labels[6], name="In-focus organoids")

<Labels layer 'In-focus organoids' at 0x1b024d8beb0>

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342
Selected labels layer: In-focus organoids
Selected measurements: ['area', 'bbox_area', 'equivalent_diameter', 'convex_area', 'perimeter', 'perimeter_crofton', 'extent', 'local_centroid-0', 'local_centroid-1', 'solidity', 'feret_diameter_max', 'major_axis_length', 'minor_axis_length', 'orientation', 'eccentricity', 'aspect_ratio', 'roundness', 'circularity']


c:\Users\alber.DESKTOP-VIFBJNG\mambaforge\envs\dbn39\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\alber.DESKTOP-VIFBJNG\mambaforge\envs\dbn39\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\alber.DESKTOP-VIFBJNG\mambaforge\envs\dbn39\lib

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342
Dimensionality reduction finished
Selected labels layer: In-focus organoids
Selected measurements: ['UMAP_0', 'UMAP_1']
Selected clustering method: KMeans
KMEANS predictions finished.
Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


In [ ]:
# TODO: Merge all labels from each well into a big image, so I can use dimensionality reduction and cluster plotter on one image.